# Filtrage passe-bas, Ã  bande Ã©troite et analyse de la TZ

*But* : Analyser les filtres Ã  bandes Ã©troites et comprendre comment les reprÃ©senter via la TZ



## 1. Filtre passe-haut

- On commence par charger le fichier du notebook prÃ©cÃ©dent

In [ ]:
%matplotlib inline
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

import soundfile as sf
import IPython.display 


x_loop, Fe = sf.read("myloop.wav")
x_loop = x_loop[:,0] #prenons qu'un seul canal 
IPython.display.Audio(x_loop, rate=Fe)



- Un filtre passe-haut amplifie, comme son nom l'indique, les hautes frÃ©quences jusqu'Ã  une certaine valeur.  

Nous allons d'abord Ã©tudier un filtre de fonction de transfert suivante : 

$$
H(z) = \frac{1-z^{-1}}{1+0.5z^{-1}}
$$

**Questions et application**:
1. VÃ©rifier ses caractÃ©ristiques (stable, causal etc. ?)
2. DÃ©terminer son pÃ´le et son zÃ©ro
3. Tracer la rÃ©ponse en frÃ©quence du filtre ainsi que les pÃ´les et zÃ©ros sur un cercle (aidez vous des fonctions ci-dessous)
4. Commentez ce que vous entendez.

In [ ]:
def reponse_en_frequence(b, a):
    w,h = sig.freqz(b,a,4096)
    hdb = 20*np.log10(np.abs(h))
    plt.plot(w/2/np.pi,hdb)
    plt.grid()
    plt.xlabel(r'frÃ©quence rÃ©duite $\nu$')
    plt.ylabel('dB')
    plt.title('RÃ©ponse en frÃ©quence du filtre')
    plt.show()



def plot_poles_and_zeros(poles, zeros):
    # Plot the poles and zeros
    plt.plot(np.real(zeros), np.imag(zeros), 'o', label='Zeros')
    plt.plot(np.real(poles), np.imag(poles), 'x', label='Poles')

    # Add axis labels and a legend
    plt.xlabel('Partie rÃ©elle')
    plt.ylabel('Partie Imaginaire')
    plt.title('PÃ´les et zÃ©ros')
    plt.legend()


    # Ajouter un cercle unitÃ©
    theta = np.linspace(0, 2*np.pi, 100)
    plt.plot(np.cos(theta), np.sin(theta), '--', color='gray')

    plt.xlim([-2.5, 2.5])
    plt.ylim([-2.5, 2.5])
    plt.gca().set_aspect('equal', adjustable='box')

    #afficher le plot
    plt.grid()
    plt.show()


In [ ]:
a = TODO #  dÃ©nominateur
b = TODO #  numÃ©rateur

x_f = sig.lfilter(b,a,x_loop)

pi = np.pi


# Calculer les pÃ´les et les zÃ©ros
zeros = np.roots(b)
poles = np.roots(a)

# Si on part des poles et des zÃ©ros, on a 
# a = np.poly(zeros)
# b = np.poly(poles)

reponse_en_frequence(b, a)

plot_poles_and_zeros(poles, zeros)

IPython.display.Audio(x_f, rate=Fe)


## 2. Filtre passe-bas

1. Commencez par inverser le dÃ©nominateur et le numÃ©rateur du filtre prÃ©cÃ©dent. C'est Ã  dire, considÃ©rez la fonction de transfert suivante : 
$$
H(z) = \frac{1+0.5z^{-1}}{1-z^{-1}}
$$
Faites la mÃªme analyse qu'avant (rÃ©ponse en frÃ©quence, poles/zÃ©ros, stabilitÃ©, causalitÃ© etc.)

2. Vous recevez un message d'erreur ? Quelle propriÃ©tÃ© n'a pas ce filtre qu'avait l'autre filtre ? 
   
3. On va remplacer par le filtre suivant : 
$$
H(z) = \frac{1+0.5z^{-1}}{1-0.95z^{-1}}
$$
Est-ce que la propriÃ©tÃ© du premier filtre (dans la section 1) manquante a Ã©tÃ© rÃ©cupÃ©rÃ©e ? 

4. Essayez de jouer avec les valeurs du pÃ´le et du zÃ©ro pour voir l'influence sur la rÃ©ponse en frÃ©quence. En conclure alors Ã  partir de cette premiÃ¨re section et de la seconde section les valeurs des points $[1, 0], [-1, 0]$ sur le cercle (au choix: $\frac{fe}{2} Hz$ ou $0$) 

# 3. Filtre Passe-bande
## 3.1. L'effet "vieux tÃ©lÃ©phone"
Les appels tÃ©lÃ©phoniques, pour des raisons techniques, Ã©taient limitÃ©s Ã  une bande frÃ©quentielle de $200 Hz â€“ 3,4 kHz$ dans le passÃ©. 

On va charger tout d'abord le fichier ``speech.wav`` et tenter d'appliquer un filtre passe-bande correspondant Ã  $200HZ - 3.4 kHz$

In [ ]:
x_speech, Fe = sf.read("speech.wav")
print(Fe)
IPython.display.Audio(x_speech, rate=Fe)

1. DÃ©terminer les angles $\theta_{\text{in}}, \theta_{\text{out}}$ sur le demi-cercle unitÃ© correspondant aux frÃ©quences $200Hz$ et $3.4 kHz$ respectivement. 
2. Soient $z_{\text{in}} = re^{i\theta_{\text{in}}}$ et $z_{\text{out}} = re^{i\theta_{\text{out}}}$ les pÃ´les correspondant aux frÃ©quences du filtre passe bande avec $r < 1$. On considÃ¨re $1$ pÃ´le $z_1$ son conjuguÃ© :
$$
H(z) = \frac{1}{|1-z_1 z^{-1}|^2}
$$
Pourquoi considÃ©rer les pÃ´les conjuguÃ©s Ã  votre avis ?

1. Application : Prenez $1$ pÃ´le entre $\theta_{\text{in}}$ et $\theta_{\text{out}}$ au centre de la bande et appliquez-le au filtre. Observez la rÃ©ponse frÃ©quentielle du filtre en fonction des paramÃ¨tres, contrÃ´lez la position des pÃ´les et concluez.
   
2. AmÃ©lioration : Rajoutez des zÃ©ros conjuguÃ©s dans $H(z)$ pour encore plus insister sur la rÃ©ponse frÃ©quentielle du filtre dans la bande passante voulue.  

In [ ]:
r_zeros = TODO
r_pole = TODO
Theta_in = TODO
Theta_out = TODO
thetas_poles = TODO
thetas_zeros = TODO

z_pole = r_pole * np.concatenate((np.exp(1j * thetas_poles), (np.exp(-1j * thetas_poles))),
                       axis=0)
z_zeros = r_zeros * np.concatenate((np.exp(1j * thetas_zeros), (np.exp(-1j * thetas_zeros))),
                       axis=0)
a = np.poly(z_pole)  # denominateur
b = np.poly(z_zeros) # numÃ©rateur

x_f = TODO

reponse_en_frequence(b, a)

plot_poles_and_zeros(poles, zeros)



## 3.2. Isolation des notes

Un modÃ¨le simple pour un instrument harmonique quand il joue une note est qu'il est composÃ© : 
- d'une frÃ©quence fondamentale (souvent appellÃ©e $f_0$)
- d'harmoniques ou *partiel harmonique* (proportionnel Ã  $f_0$ *e.g.* $2 f_0, 3f_0 \dots$)

Les frÃ©quences fondamentales du piano sont disponibles sur Wikipedia [ici](https://fr.wikipedia.org/wiki/Fr%C3%A9quences_des_touches_du_piano)
Le la est Ã  440 Hz. Quand la frÃ©quence double, la note porte le mÃªme nom mais on change d'octave. Le rapport de frÃ©quence entre deux notes successives est Ã©gal Ã  2**(1/12).

On s'intÃ©resse tout d'abord Ã  un fichier gÃ©nÃ©rÃ© avec des sinusoÃ¯des. Nous allons crÃ©er un filtre  stable et causal qui rÃ©sonne quand une note donnÃ©e est prÃ©sente dans le signal mais pas pour les autres notes. On pourra 
- soit placer les pÃ´les et les zÃ©ros de maniÃ¨re Ã  obtenir l'effet voulu, 
- soit partir d'un filtre passe-bas et dÃ©caler sa rÃ©ponse en frÃ©quence de maniÃ¨re appropriÃ©e.


Conseil: Il peut Ãªtre intÃ©ressant de retourner le module d'un filtre complexe. En effet la sortie d'un filtre rÃ©el oscillera Ã  la frÃ©quence recherchÃ©e, ce qui gÃªne la visualisation.

In [ ]:
t = np.arange(0, 5, 1/Fe)
son = np.concatenate([np.sin(2 * np.pi * t[0:len(t)//3] * 440), 
                      np.sin(2*np.pi*t[len(t)//3:2*len(t)//3] * 440*2**(1/12)), 
                      np.sin(2*np.pi*t[2*len(t)//3:] * 440 * 2) ])

In [ ]:
IPython.display.Audio(son, rate=Fe)

On charge ci-aprÃ¨s le fichier ``Piano_accord.wav`` qui est composÃ© des notes $\texttt{Do 4, Mi 4, Sol 4} $. Essayez de faire de mÃªme avec ce morceau plus complexe.

In [ ]:
x_accord, Fe = sf.read("Piano_accord.wav")
x_accord = x_accord[:,0] #prenons qu'un seul canal 
IPython.display.Audio(x_accord, rate=Fe)

## 3.3. TransformÃ©e de Fourier Ã  court terme
Montrez que la TCFT peut se rÃ©interprÃ©ter comme un ensemble de filtres dÃ©tecteurs de notes. En dÃ©duire un algorithme rapide pour dÃ©tecter un grand nombre de notes en mÃªme temps.
